In [1]:
import requests
import pandas as pd
import io
import time
import datetime
import numpy as np
import json

In [2]:
TWSE_list = pd.DataFrame()
def Crawler_TWSE_List():
  global TWSE_list
  url = 'https://isin.twse.com.tw/isin/C_public.jsp?strMode=2'
  page = requests.get(url)
  TWSE_list = pd.read_html(page.text)[0]
  TWSE_list.columns = TWSE_list.iloc[0]
  end = 0
  for i in range(0,len(TWSE_list)):
    if (TWSE_list.at[i, '有價證券代號及名稱'][0:4] == '上市認購'):
        end = i
  TWSE_list = TWSE_list.iloc[2:end]
  TWSE_list = TWSE_list[['有價證券代號及名稱', '國際證券辨識號碼(ISIN Code)']]
  TWSE_list  = TWSE_list.rename(columns={'有價證券代號及名稱': 'Stockcode', '國際證券辨識號碼(ISIN Code)': 'Name'})
  TWSE_list.reset_index(drop=True, inplace=True)
  for i in range (0,len(TWSE_list)):
    TWSE_list.at[i, 'Name'] = TWSE_list.at[i, 'Stockcode'][5:]
    TWSE_list.at[i, 'Stockcode'] = TWSE_list.at[i, 'Stockcode'][0:4]

In [3]:
TPEx_list = pd.DataFrame()
def Crawler_TPEx_List():
  global TPEx_list
  url = 'https://isin.twse.com.tw/isin/C_public.jsp?strMode=4'
  page = requests.get(url)
  TPEx_list = pd.read_html(page.text)[0]
  TPEx_list.columns = TPEx_list.iloc[0]
  start = 0
  end = 0
  for i in range(0,len(TPEx_list)):
    if (TPEx_list.at[i, '有價證券代號及名稱'][0:4] == '股票'):
        start = i+1
    if (TPEx_list.at[i, '有價證券代號及名稱'][0:4] == '特別股'):
        end = i
  TPEx_list = TPEx_list.iloc[start:end]
  TPEx_list = TPEx_list[['有價證券代號及名稱', '國際證券辨識號碼(ISIN Code)']]
  TPEx_list = TPEx_list.rename(columns={'有價證券代號及名稱': 'Stockcode', '國際證券辨識號碼(ISIN Code)': 'Name'})
  TPEx_list.reset_index(drop=True, inplace=True)
  for i in range (0,len(TPEx_list)):
    TPEx_list.at[i, 'Name'] = TPEx_list.at[i, 'Stockcode'][5:]
    TPEx_list.at[i, 'Stockcode'] = TPEx_list.at[i, 'Stockcode'][0:4]

In [4]:
StockPrice_TWSE = pd.read_excel('Stock_TWSE.xlsx', header=[0,1], index_col=0)
StockPrice_TPEx = pd.read_excel('Stock_TPEx.xlsx', header=[0,1], index_col=0)

In [8]:
different_Tpex = None
different_TWSE = None
def Update_Stock_List():
  global different_Tpex
  global different_TWSE
  Crawler_TPEx_List()
  Crawler_TWSE_List()
  if list(dict.fromkeys(StockPrice_TPEx.columns.get_level_values(0)[1:])) == list(TPEx_list.iloc[:,0]):
    print('TPEx Pass')
  else:
    if (len(TPEx_list) > (len(dict.fromkeys(StockPrice_TPEx.columns.get_level_values(0)[1:]))-1)):
        different_Tpex = set(TPEx_list.iloc[:,0]) - set(StockPrice_TPEx.columns.get_level_values(0)[1:])
        different_Tpex = list(different_Tpex)
        print(different_Tpex)
        for i in range(0,len(different_Tpex)):
            StockPrice_TPEx.insert(len(StockPrice_TPEx.columns), (different_Tpex[i], 'Close'), None)
            StockPrice_TPEx.insert(len(StockPrice_TPEx.columns), (different_Tpex[i], 'Volume'), None)
            StockPrice_TPEx.insert(len(StockPrice_TPEx.columns), (different_Tpex[i], 'Foundation'), None)
            StockPrice_TPEx.insert(len(StockPrice_TPEx.columns), (different_Tpex[i], 'Yield'), None)
            StockPrice_TPEx.sort_index(axis = 1, level=0, inplace=True, sort_remaining = False)
            temp = StockPrice_TPEx.pop(('Date',       'Date'))
            StockPrice_TPEx.insert(0, ('Date',       'Date'), temp)
    else:
        different_Tpex = set(StockPrice_TPEx.columns.get_level_values(0)[1:]) - set(TPEx_list.iloc[:,0])
        different_Tpex = list(different_Tpex)
        print(different_Tpex)
        for i in range(0,len(different_Tpex)):
            StockPrice_TPEx.drop(columns=[different_Tpex[i]], inplace=True)
            
  if list(dict.fromkeys(StockPrice_TWSE.columns.get_level_values(0)[1:])) == list(TWSE_list.iloc[:,0]):
    print('TWSE Pass')
  else:
    if (len(TWSE_list) > (len(dict.fromkeys(StockPrice_TWSE.columns.get_level_values(0)[1:]))-1)):
        different_TWSE = set(TWSE_list.iloc[:,0]) - set(StockPrice_TWSE.columns.get_level_values(0)[1:])
        different_TWSE = list(different_TWSE)
        print(different_TWSE)
        for i in range(0,len(different_TWSE)):
            StockPrice_TWSE.insert(len(StockPrice_TWSE.columns), (different_TWSE[i], 'Close'), None)
            StockPrice_TWSE.insert(len(StockPrice_TWSE.columns), (different_TWSE[i], 'Volume'), None)
            StockPrice_TWSE.insert(len(StockPrice_TWSE.columns), (different_TWSE[i], 'Foundation'), None)
            StockPrice_TWSE.insert(len(StockPrice_TWSE.columns), (different_TWSE[i], 'Yield'), None)
            StockPrice_TWSE.sort_index(axis = 1, level=0, inplace=True, sort_remaining = False)
            temp = StockPrice_TWSE.pop(('Date',       'Date'))
            StockPrice_TWSE.insert(0, ('Date',       'Date'), temp)
    else:
        different_TWSE = set(StockPrice_TWSE.columns.get_level_values(0)[1:]) - set(TWSE_list.iloc[:,0])
        different_TWSE = list(different_TWSE)
        print(different_TWSE)
        for i in range(0,len(different_TWSE)):
            StockPrice_TWSE.drop(columns=[different_TWSE[i]], inplace=True)

In [9]:
def crawler(datetime):
    url = 'https://www.tpex.org.tw/web/stock/aftertrading/otc_quotes_no1430/stk_wn1430_result.php?l=zh-tw&o=csv&d='+ datetime +'&se=EW&s=0,asc,0'
    page = requests.get(url)
    use_text = page.text.splitlines()
    for i,text in enumerate(use_text):
        if '代號' in text:
            initial_point = i
            break
    test_df = pd.read_csv(io.StringIO(''.join([text+ '\n' for text in use_text[initial_point:]])))
    for i in range(0,len(test_df)):
        try:
            if (len(test_df.loc[i,'代號']) != 4):
                test_df.drop(i,inplace=True)
        except TypeError:
            if test_df.loc[i,'代號'] < 1200 and test_df.loc[i,'代號'] > 9999:
                test_df.drop(i,inplace=True)
    test_df = test_df[['代號', '收盤 ', '成交股數  ']]
    return test_df

def crawler_Foundation(datetime):
    url = 'https://www.tpex.org.tw/web/stock/3insti/daily_trade/3itrade_hedge_result.php?l=zh-tw&o=csv&se=EW&t=D&d='+ datetime +'&s=0,asc'
    page = requests.get(url)
    use_text = page.text.splitlines()
    for i,text in enumerate(use_text):
        if '代號' in text:
            initial_point = i
            break
    test_df = pd.read_csv(io.StringIO(''.join([text+ '\n' for text in use_text[initial_point:]])))
    if len(test_df) > 1:
        for i in range(0,len(test_df)):
            try:
                if (len(test_df.loc[i,'代號']) != 4):
                    test_df.drop(i,inplace=True)
            except TypeError:
                if test_df.loc[i,'代號'] < 1200 and test_df.loc[i,'代號'] > 9999:
                    test_df.drop(i,inplace=True)
        try:
            test_df = test_df[['代號', '三大法人買賣超股數合計']]
        except KeyError:
            test_df = test_df[['代號', '三大法人買賣超股數']]
        test_df.reset_index(drop=True,inplace=False)
    else:
        del test_df
    return test_df

def crawler_Yield(datetime):
    url = 'https://www.tpex.org.tw/web/stock/aftertrading/peratio_analysis/pera_result.php?l=zh-tw&o=csv&charset=UTF-8&d='+datetime+'&c=&s=0,asc'
    page = requests.get(url)
    use_text = page.text.splitlines()
    for i,text in enumerate(use_text):
            if '股票代號' in text:
                initial_point = i
                break
    test_df = pd.read_csv(io.StringIO(''.join([text+ '\n' for text in use_text[initial_point:]])))
    if (len(use_text) < 10):
        del test_df
    else:
        test_df = test_df[['股票代號', '殖利率(%)']]
        test_df = test_df.rename(columns={"股票代號": '代號'})
    return test_df

#Day_TPEx = []
def trans_date(datetime):
#    global Day_TPEx
    d = datetime.day
    m = datetime.month
    y = datetime.year-1911
    if m < 10:
        if d < 10:
            Day_TPEx = (str(y)+'/0'+str(m)+'/0'+str(d))
        else:
            Day_TPEx = (str(y)+'/0'+str(m)+'/'+str(d))
    else:
        if d < 10:
            Day_TPEx = (str(y)+'/'+str(m)+'/0'+str(d))
        else:
            Day_TPEx = (str(y)+'/'+str(m)+'/'+str(d))
            
    return Day_TPEx

Crawler_Price = pd.DataFrame()
Crawler_Foundation = pd.DataFrame()
Crawler_Yield = pd.DataFrame()
def crawler_data(datetime):
    try:
        global Crawler_Price
        global Crawler_Foundation
        global Crawler_Yield
        Crawler_Price = crawler(trans_date(datetime))
        Crawler_Foundation = crawler_Foundation(trans_date(datetime))
        Crawler_Yield = crawler_Yield(trans_date(datetime))
        print('TPEx Crawler Successful')
    except UnboundLocalError:
        print('TPEx Crawler Fail')
    except KeyError:
        print('TPEx Crawler Fail')

In [10]:
def catch(datetime):
    url = 'https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX?date='+datetime+'&type=ALLBUT0999&response=json'
    page = requests.get(url).text
    json_data = json.loads(page)
    Stock_data = json_data['tables'][8]['data']
    test_df = pd.DataFrame(Stock_data, columns = [json_data['tables'][8]['fields']])
    for i in range(0,len(test_df)):
        if ((len(test_df.loc[i,'證券代號'][0])) != 4):
            test_df.drop(i,inplace=True)
        elif int(test_df.loc[i,'證券代號'][0]) < 1100 or int(test_df.loc[i,'證券代號'][0]) > 9999:
            test_df.drop(i,inplace=True)
    temp = test_df[['證券代號', '收盤價', '成交股數']]
    test_df = pd.DataFrame(temp.values, columns = [temp.columns[0][0],temp.columns[1][0],temp.columns[2][0]])
    return test_df

def catch_Foundation(datetime):
    url = 'https://www.twse.com.tw/rwd/zh/fund/T86?date='+datetime+'&selectType=ALLBUT0999&response=json'
    page = requests.get(url).text
    json_data = json.loads(page)
    Stock_data = json_data['data']
    test_df = pd.DataFrame(Stock_data, columns = [json_data['fields']])
    for i in range(0,len(test_df)):
        if ((len(test_df.loc[i,'證券代號'][0])) != 4):
            test_df.drop(i,inplace=True)
        elif int(test_df.loc[i,'證券代號'][0]) < 1100 or int(test_df.loc[i,'證券代號'][0]) > 9999:
            test_df.drop(i,inplace=True)
    temp = test_df[['證券代號', '三大法人買賣超股數']]
    test_df = pd.DataFrame(temp.values, columns = [temp.columns[0][0],temp.columns[1][0]])
    test_df.sort_values(by=['證券代號'], inplace=True)
    test_df.reset_index(drop=True,inplace=True)
    return test_df

def catch_Yield(datetime):
    url = 'https://www.twse.com.tw/rwd/zh/afterTrading/BWIBBU_d?date='+datetime+'&selectType=ALL&response=json'
    page = requests.get(url).text
    json_data = json.loads(page)
    Stock_data = json_data['data']
    test_df = pd.DataFrame(Stock_data, columns = [json_data['fields']])
    temp = test_df[['證券代號', '殖利率(%)']]
    test_df = pd.DataFrame(temp.values, columns = [temp.columns[0][0],temp.columns[1][0]])
    return test_df
    
def catch_day(datetime):
    d = datetime.day
    m = datetime.month
    y = datetime.year
    if m < 10:
        if d < 10:
            Day_TWSE = str(y)+'0'+str(m)+'0'+str(d)
        else:
            Day_TWSE = str(y)+'0'+str(m)+str(d)
    else:
        if d < 10:
            Day_TWSE = str(y)+str(m)+'0'+str(d)
        else:
            Day_TWSE = str(y)+str(m)+str(d)
            
    return Day_TWSE

Catch_Price = pd.DataFrame()
Catch_Foundation = pd.DataFrame()
Catch_Yield = pd.DataFrame()
def catch_data(datetime):
    try:
        global Catch_Price
        global Catch_Foundation
        global Catch_Yield
        Catch_Price = catch(catch_day(datetime))
        Catch_Foundation = catch_Foundation(catch_day(datetime))
        Catch_Yield = catch_Yield(catch_day(datetime))
        print('TWSE Crawler Successful')
    except UnboundLocalError:
        print('TWSE Crawler Fail')
    except KeyError:
        print('TWSE Crawler Fail')

In [11]:
Crawler_Frame = pd.DataFrame()
Catch_Frame = pd.DataFrame()
def Get_New_Data(datetime):
    start = 0
    global Crawler_Frame
    global Crawler_Price
    global Crawler_Foundation
    global Crawler_Yield
    global Catch_Frame
    global Catch_Price
    global Catch_Foundation
    global Catch_Yield
    crawler_data(datetime)
    Crawler_Frame = pd.concat([Crawler_Price.set_index('代號'), Crawler_Foundation.set_index('代號').reindex(Crawler_Price.set_index('代號').index), Crawler_Yield.set_index('代號').reindex(Crawler_Price.set_index('代號').index)], axis=1)
    Crawler_Frame = Crawler_Frame.reset_index()
    catch_data(datetime)
    Catch_Frame = pd.concat([Catch_Price.set_index('證券代號'), Catch_Foundation.set_index('證券代號').reindex(Catch_Price.set_index('證券代號').index), Catch_Yield.set_index('證券代號').reindex(Catch_Price.set_index('證券代號').index)], axis=1)
    Catch_Frame = Catch_Frame.reset_index()

In [12]:
def Time_transfer(datetime):
  d = datetime.day
  m = datetime.month
  y = datetime.year
  if m < 10:
    if d < 10:
      Time = str(y)+'-0'+str(m)+'-0'+str(d)
    else:
      Time = str(y)+'-0'+str(m)+'-'+str(d)
  else:
    if d < 10:
      Time = str(y)+'-'+str(m)+'-0'+str(d)
    else:
      Time = str(y)+'-'+str(m)+'-'+str(d)
  return Time

In [14]:
def Update_Stock_Price(datetime):
  temp = 0
  try:
    url = 'https://www.tpex.org.tw/web/stock/aftertrading/otc_quotes_no1430/stk_wn1430_result.php?l=zh-tw&o=csv&d='+trans_date(datetime)+'&se=EW&s=0,asc,0'
    page = requests.get(url)
    use_text = page.text.splitlines()
    for i,text in enumerate(use_text):
      if '代號' in text:
        initial_point = i
        break
    test_df = pd.read_csv(io.StringIO(''.join([text+ '\n' for text in use_text[initial_point:]])))
  except NameError:
    print('No New Trading Data')
    temp = 1
    
#  global different_Tpex
#  global different_TWSE
#  Update_Stock_List()
                            
  if temp == 0:
    Update_Stock_List()
    Get_New_Data(datetime)
    global StockPrice_TWSE
    global StockPrice_TPEx
    temp1 = 0
    temp2 = 0
    if StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,0] == Time_transfer(datetime):
      print('TWSE Exists!')
      temp1 = 1
    else:
      StockPrice_TWSE = StockPrice_TWSE.append({('Date',       'Date'):Time_transfer(datetime)}, ignore_index=True)
    
    if StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,0] == Time_transfer(datetime):
      print('TPEx Exists!')
      temp2 = 1
    else:
      StockPrice_TPEx = StockPrice_TPEx.append({('Date',       'Date'):Time_transfer(datetime)}, ignore_index=True)
    
    if temp2 == 0:
      for i in range (0,len(Crawler_Frame)):
        for j in range (1, len(list(StockPrice_TPEx)),4):
          if (Crawler_Frame.loc[i,'代號'] == list(StockPrice_TPEx)[j][0]):
            try:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j] = float(Crawler_Frame.iloc[i,1])
            except ValueError:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j] = None
            
            try:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+1] = float(Crawler_Frame.iloc[i,2].replace(',',''))
            except ValueError:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+1] = None
            except AttributeError:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+1] = None
            
            try:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+2] = float(Crawler_Frame.iloc[i,3].replace(',',''))
            except ValueError:              
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+2] = None
            except AttributeError:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+1] = None
            
            try:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+3] = float(Crawler_Frame.iloc[i,4])
            except ValueError:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+3] = None
    
    if temp1 == 0:
      for x in range (0,len(Catch_Frame)):
        for y in range (1, len(list(StockPrice_TWSE)),4):
          if (Catch_Frame.loc[x,'證券代號'] == list(StockPrice_TWSE)[y][0]):
            try:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y] = float(Catch_Frame.iloc[x,1])
            except ValueError:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y] = None
            
            try:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+1] = float(Catch_Frame.iloc[x,2].replace(',',''))
            except ValueError:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+1] = None
            except AttributeError:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+1] = None
            
            try:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+2] = float(Catch_Frame.iloc[x,3].replace(',',''))
            except ValueError:              
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+2] = None
            except AttributeError:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+1] = None
            
            try:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+3] = float(Catch_Frame.iloc[x,4])
            except ValueError:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+3] = None
            
    print('Finished!')  

In [20]:
Update_Stock_Price(datetime.datetime(2023, 3, 30, 10, 1, 55, 280233))

TPEx Pass
TWSE Pass
TPEx Crawler Successful
TWSE Crawler Successful


C:\Users\Richie\AppData\Local\Temp\ipykernel_6588\3166547493.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  StockPrice_TWSE = StockPrice_TWSE.append({('Date',       'Date'):Time_transfer(datetime)}, ignore_index=True)
C:\Users\Richie\AppData\Local\Temp\ipykernel_6588\3166547493.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  StockPrice_TPEx = StockPrice_TPEx.append({('Date',       'Date'):Time_transfer(datetime)}, ignore_index=True)


Finished!


In [21]:
StockPrice_TWSE

Date   1101                                1102             \
            Date  Close      Volume Foundation Yield  Close     Volume   
0     2016-01-04  26.15  10437306.0 -2717840.0  9.52  26.25  6852789.0   
1     2016-01-05  26.40   5382070.0  1819649.0  9.43  26.00  3967822.0   
2     2016-01-06  26.05   5154746.0  -135962.0  9.56  25.95  3736159.0   
3     2016-01-07  27.35  20289500.0  8650064.0  9.10  26.15  5483126.0   
4     2016-01-08  27.00  11915555.0 -3786818.0  9.22  26.00  5233511.0   
...          ...    ...         ...        ...   ...    ...        ...   
1762  2023-03-24  37.60  13819470.0   417486.0  5.32  43.00  3514559.0   
1763  2023-03-27  36.65  25738990.0 -4132184.0  5.46  42.90  3915506.0   
1764  2023-03-28  36.20  27286673.0 -4029347.0  1.38  43.10  3607083.0   
1765  2023-03-29  36.35  12924440.0  -291933.0  1.38  43.30  4326952.0   
1766  2023-03-30  36.20  12765508.0 -4693165.0  1.38  43.20  3385564.0   

                        1103  ...       9946         9955             \
     Foundation Yield  Close  ... Foundation Yield  Close     Volume   
0    -2582600.0  8.38   9.40  ...    11000.0  3.09  12.15    77112.0   
1     -390654.0  8.46   9.35  ...   -20000.0  3.13  12.20    74106.0   
2      -27353.0  8.48   9.27  ...   -39000.0  3.20  12.10    25100.0   
3     -142190.0  8.41   9.26  ...   -47365.0  3.42  12.20   124337.0   
4    -2929390.0  8.46   9.23  ...     2000.0  3.43  12.50   207000.0   
...         ...   ...    ...  ...        ...   ...    ...        ...   
1762  -995860.0  5.35  19.25  ...     1000.0  5.74  26.25  3006637.0   
1763  -490134.0  5.36  19.05  ...        0.0  5.74  25.65  1434085.0   
1764    65526.0  5.34  18.85  ...    -1000.0  5.71  25.35  1617015.0   
1765  -459657.0  5.31  18.90  ...     1000.0  5.71  25.15   923664.0   
1766 -1324369.0  5.32  18.95  ...    -1000.0  5.71  24.90   941387.0   

                        9958                              
     Foundation Yield  Close     Volume Foundation Yield  
0        4000.0   0.0   9.95    56147.0    -7000.0  3.02  
1      -19000.0   0.0   9.95   101434.0   -20000.0  3.02  
2           NaN   0.0   9.84    71995.0        NaN  3.05  
3        2337.0   0.0   9.78   112830.0    -7000.0  3.07  
4      -23000.0   0.0   9.86   101907.0    39000.0  3.04  
...         ...   ...    ...        ...        ...   ...  
1762   -24000.0   0.0  99.00  1148946.0  -100271.0  3.02  
1763    -4100.0   0.0  97.30  1411198.0  -309436.0  3.07  
1764   261000.0   0.0  96.50  1215409.0   -44565.0  3.10  
1765    26000.0   0.0  93.60  3954235.0  -901320.0  3.19  
1766   208000.0   0.0  92.40  4473031.0   472151.0  3.24  

[1767 rows x 3901 columns]

In [22]:
StockPrice_TPEx

Date   1240                             1258                       \
            Date  Close   Volume Foundation Yield  Close    Volume Foundation   
0     2016-01-04    NaN      NaN        NaN   NaN  39.50   51000.0        NaN   
1     2016-01-05    NaN      NaN        NaN   NaN  38.90  116000.0        NaN   
2     2016-01-06    NaN      NaN        NaN   NaN  38.05   85000.0        NaN   
3     2016-01-07    NaN      NaN        NaN   NaN  36.50  109000.0        NaN   
4     2016-01-08    NaN      NaN        NaN   NaN  37.30   59000.0        NaN   
...          ...    ...      ...        ...   ...    ...       ...        ...   
1762  2023-03-24  45.10  40000.0     1000.0  5.54  26.65  174000.0    -1000.0   
1763  2023-03-27  45.60  38000.0     8000.0  5.48  26.65       NaN        NaN   
1764  2023-03-28  44.70  55000.0    -8000.0  2.68  26.65       NaN        NaN   
1765  2023-03-29  44.85  41000.0     3000.0  2.68  26.65       NaN        NaN   
1766  2023-03-30  44.90   9000.0     3000.0  2.67  26.70   33000.0     2000.0   

            1259  ...       9951         9960                             \
     Yield Close  ... Foundation Yield  Close    Volume Foundation Yield   
0     3.29  72.1  ...        NaN  3.33  19.00    2000.0        NaN  4.21   
1     3.34  70.9  ...        NaN  3.42  18.90    4000.0        NaN  4.23   
2     3.42  70.3  ...        NaN  3.48  18.70    2000.0        NaN  4.28   
3     3.56  69.1  ...        NaN  3.54  18.85   78000.0        NaN  4.24   
4     3.49  71.8  ...        NaN  3.41  18.75   83000.0        NaN  4.27   
...    ...   ...  ...        ...   ...    ...       ...        ...   ...   
1762  0.00  74.4  ...   -86000.0  4.63  29.95  134000.0     4000.0  7.35   
1763  0.00  74.5  ...     4000.0  4.59  30.75  179000.0    13000.0  7.15   
1764  0.00  74.5  ...    -7000.0  4.73  30.60   28000.0    -1000.0  7.19   
1765  0.00  74.0  ...    -6000.0  4.72  30.80   50000.0     1000.0  7.14   
1766  0.00  74.0  ...    15000.0  4.67  30.70   18000.0    -2000.0  7.17   

       9962                              
      Close     Volume Foundation Yield  
0      7.25     1000.0        NaN  2.76  
1      7.17     2000.0        NaN  2.79  
2      7.06    35000.0        NaN  2.83  
3      7.07    30000.0        NaN  2.83  
4      7.06     1000.0        NaN  2.83  
...     ...        ...        ...   ...  
1762  21.25  1824000.0  -567000.0  8.47  
1763  21.55   993000.0   225000.0  8.35  
1764  21.35   961000.0  -402000.0  8.43  
1765  21.15  1008000.0  -359000.0  8.51  
1766  21.40   797000.0   -79000.0  8.41  

[1767 rows x 3241 columns]

In [23]:
StockPrice_TWSE.to_excel('Stock_TWSE.xlsx')
StockPrice_TPEx.to_excel('Stock_TPEx.xlsx')

In [24]:
MA_TWSE = pd.DataFrame(index=['7MA', '14MA', '30MA', '60MA', '90MA', '180MA', '360MA', '720MA', '1080MA'],columns=StockPrice_TWSE.columns.get_level_values(0)[1:].drop_duplicates())
MA_TWSE.index.name ='Moving Average'
MA_TPEx = pd.DataFrame(index=['7MA', '14MA', '30MA', '60MA', '90MA', '180MA', '360MA', '720MA', '1080MA'],columns=StockPrice_TPEx.columns.get_level_values(0)[1:].drop_duplicates())
MA_TPEx.index.name ='Moving Average'
for i in range(0,len(MA_TWSE.columns)):
    for j in range(0,len(MA_TWSE)):
        MA_TWSE.iloc[j,i] = round(((StockPrice_TWSE.iloc[int('-' + MA_TWSE.index.str.extract('(\d+)',expand=False)[j]):,i*4+1] * StockPrice_TWSE.iloc[int('-' + MA_TWSE.index.str.extract('(\d+)',expand=False)[j]):,i*4+2])/(StockPrice_TWSE.iloc[int('-' + MA_TWSE.index.str.extract('(\d+)',expand=False)[j]):,i*4+2].mean())).mean(), 2)
    
for i in range(0,len(MA_TPEx.columns)):
    for j in range(0,len(MA_TPEx)):
        MA_TPEx.iloc[j,i] = round(((StockPrice_TPEx.iloc[int('-' + MA_TPEx.index.str.extract('(\d+)',expand=False)[j]):,i*4+1] * StockPrice_TPEx.iloc[int('-' + MA_TPEx.index.str.extract('(\d+)',expand=False)[j]):,i*4+2])/(StockPrice_TPEx.iloc[int('-' + MA_TPEx.index.str.extract('(\d+)',expand=False)[j]):,i*4+2].mean())).mean(), 2)

In [25]:
MA_TWSE

,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,...,9939,9940,9941,9942,9943,9944,9945,9946,9955,9958
Moving Average,,,,,,,,,,,,,,,,,,,,,
7MA,36.74,43.06,19.02,25.96,14.07,17.03,17.65,19.99,38.45,50.13,...,97.46,28.76,177.5,117.92,68.08,20.37,35.54,11.89,25.91,95.16
14MA,36.71,42.88,18.91,25.49,13.83,16.98,17.75,20.19,37.88,50.11,...,98.3,28.63,175.92,117.05,67.3,20.36,36.08,11.9,25.0,97.22
30MA,37.6,43.34,20.19,25.18,13.75,16.98,17.8,20.59,37.3,50.33,...,98.4,28.65,175.54,119.4,68.7,20.42,38.06,12.0,24.67,100.9
60MA,36.6,42.58,19.87,24.62,13.53,16.69,17.96,20.24,36.79,49.07,...,95.29,27.94,170.61,114.76,66.47,20.43,39.87,12.01,24.0,97.0
90MA,35.21,41.83,19.46,24.05,13.33,16.42,18.08,19.97,36.09,48.26,...,92.31,28.03,165.98,113.05,65.54,20.46,41.42,12.04,23.43,92.83
180MA,35.69,41.85,18.8,23.12,12.97,16.72,18.07,19.83,34.54,47.51,...,87.05,29.14,163.72,107.16,62.18,20.37,50.68,12.53,22.4,88.92
360MA,39.28,43.2,19.69,22.19,12.64,19.15,20.15,21.59,33.89,50.4,...,81.71,32.71,179.11,103.9,60.29,21.63,60.31,13.66,25.49,105.52
720MA,43.25,44.92,20.71,22.73,13.61,22.28,19.37,21.79,33.87,50.47,...,73.46,31.16,157.16,96.89,62.88,22.54,56.62,16.02,23.98,111.01
1080MA,42.43,43.66,19.4,21.99,13.32,20.63,19.07,24.13,31.09,46.43,...,68.56,30.94,145.19,93.4,63.36,22.36,54.99,20.5,22.52,99.56


In [26]:
MA_TPEx

,1240,1258,1259,1264,1268,1336,1565,1569,1570,1580,...,8936,8937,8938,8941,8942,9949,9950,9951,9960,9962
Moving Average,,,,,,,,,,,,,,,,,,,,,
7MA,45.07,26.66,74.33,267.29,135.09,27.66,260.78,25.54,17.85,101.04,...,28.28,15.09,106.19,58.71,66.44,22.84,11.5,71.11,30.35,21.42
14MA,45.18,26.65,75.85,267.68,132.18,27.8,260.64,25.32,18.09,100.69,...,28.52,15.98,107.8,59.71,66.46,22.39,11.88,71.03,29.49,21.5
30MA,47.44,27.51,78.01,269.95,144.97,29.03,262.28,25.07,18.15,99.83,...,29.43,15.65,109.88,60.05,66.8,22.08,13.21,71.18,28.34,21.4
60MA,47.14,25.7,78.41,267.44,141.67,28.97,258.37,24.57,17.85,95.97,...,28.94,15.52,106.92,58.92,64.69,20.91,12.64,69.47,27.4,19.95
90MA,46.51,23.84,77.42,264.48,138.57,28.15,253.54,24.24,17.65,93.33,...,28.39,15.34,105.47,58.13,63.54,20.64,12.18,67.44,26.83,19.42
180MA,45.87,24.77,74.67,262.11,136.57,27.09,250.41,25.0,17.2,92.72,...,27.22,14.69,100.88,57.54,63.02,20.21,12.27,65.82,26.58,17.94
360MA,59.16,24.62,72.55,273.19,134.07,26.46,273.28,25.76,21.95,98.52,...,25.91,14.48,93.89,53.99,65.71,16.01,11.92,70.47,26.72,18.91
720MA,58.86,23.67,70.42,263.7,133.51,22.83,312.14,28.32,20.04,95.16,...,28.09,15.91,78.42,51.21,65.3,14.01,11.07,86.85,27.69,19.34
1080MA,55.82,23.45,70.37,244.4,133.21,21.99,355.3,26.37,19.68,98.7,...,27.11,15.89,74.5,50.29,66.09,13.17,12.01,88.61,28.44,19.16


In [702]:
temp = [['<7MA', '<14MA', '<30MA', '<60MA', '<90MA', '<180MA', '<360MA', '<720MA', '<1080MA'], ['>3', '>6', '>9']]
Result_TWSE = pd.DataFrame(columns=pd.MultiIndex.from_product(temp))
Result_TPEx = pd.DataFrame(columns=pd.MultiIndex.from_product(temp))
def Judgment(condition):
  global Result_TWSE
  global Result_TPEx
  if (condition == '7MA'):
    x = 0
  elif (condition == '14MA'):
    x = 1
  elif (condition == '30MA'):
    x = 2
  elif (condition == '60MA'):
    x = 3
  elif (condition == '90MA'):
    x = 4
  elif (condition == '180MA'):
    x = 5
  elif (condition == '360MA'):
    x = 6
  elif (condition == '720MA'):
    x = 7
  elif (condition == '1080MA'):
    x = 8
  Pass_0 = []
  Pass_1 = []
  Pass_2 = []
  for i in range(0,len(list(MA_TWSE))):
    if (MA_TWSE.iloc[x,i] > StockPrice_TWSE.iloc[-1,i*4+1]):
        if ((StockPrice_TWSE.iloc[-1,i*4+4] >= 3) and (StockPrice_TWSE.iloc[-1,i*4+4] < 6)):
            Pass_0.append(MA_TWSE.columns[i])
        elif ((StockPrice_TWSE.iloc[-1,i*4+4] >= 6) and (StockPrice_TWSE.iloc[-1,i*4+4] < 9)):
            Pass_1.append(MA_TWSE.columns[i])
        elif ((StockPrice_TWSE.iloc[-1,i*4+4] >= 9)):
            Pass_2.append(MA_TWSE.columns[i])
  Result_TWSE.iloc[:,x*3] = pd.Series(Pass_0)
  Result_TWSE.iloc[:,x*3+1] = pd.Series(Pass_1)
  Result_TWSE.iloc[:,x*3+2] = pd.Series(Pass_2)
  
  Pass_0 = []
  Pass_1 = []
  Pass_2 = []
  for i in range(0,len(list(MA_TPEx))):
    if (MA_TPEx.iloc[x,i] > StockPrice_TPEx.iloc[-1,i*4+1]):
        if ((StockPrice_TPEx.iloc[-1,i*4+4] >= 3) and (StockPrice_TPEx.iloc[-1,i*4+4] < 6)):
            Pass_0.append(MA_TPEx.columns[i])
        elif ((StockPrice_TPEx.iloc[-1,i*4+4] >= 6) and (StockPrice_TPEx.iloc[-1,i*4+4] < 9)):
            Pass_1.append(MA_TPEx.columns[i])
        elif ((StockPrice_TPEx.iloc[-1,i*4+4] >= 9)):
            Pass_2.append(MA_TPEx.columns[i])
  Result_TPEx.iloc[:,x*3] = pd.Series(Pass_0)
  Result_TPEx.iloc[:,x*3+1] = pd.Series(Pass_1)
  Result_TPEx.iloc[:,x*3+2] = pd.Series(Pass_2)

In [703]:
for i in range(0,len(MA_TWSE.index)):
    Judgment(MA_TWSE.index[i])
for i in range(0,len(MA_TPEx.index)):
    Judgment(MA_TPEx.index[i])
#Result_TWSE = Result_TWSE.reindex(sorted(Result_TWSE.columns), axis=1)
#Result_TPEx = Result_TPEx.reindex(sorted(Result_TPEx.columns), axis=1)

In [704]:
Result_TWSE

<7MA             <14MA             <30MA             <60MA  ... <180MA  \
      >3    >6    >9    >3    >6    >9    >3    >6    >9    >3  ...     >9   
0   1227  1341  1436  1102  1315  1312  1101  1315  1312  1103  ...   1436   
1   1232  1451  1802  1103  1325  1456  1102  1325  1456  1218  ...   1456   
2   1342  1535  2027  1210  1339  1515  1103  1712  1515  1227  ...   1515   
3   1402  1712  2540  1215  1341  1802  1215  2023  1612  1235  ...   1802   
4   1409  2023  2545  1217  1451  1806  1218  2038  1802  1314  ...   1806   
5   1598  2038  2603  1218  1526  2236  1219  2109  1806  1402  ...   2348   
6   1605  2109  2609  1219  1535  2489  1227  2114  2236  1409  ...   2489   
7   1609  2211  2636  1220  1615  2537  1232  2301  2348  1438  ...   2537   
8   1626  2301  3056  1227  1712  2540  1235  2340  2489  1457  ...   2540   
9   1720  2430  3545  1232  2023  2542  1236  2390  2537  1473  ...   2542   
10  1776  2496  5546  1235  2038  2545  1314  2477  2540  1476  ...   2609   
11  1786  2597  6807  1236  2109  2603  1323  2478  2542  1517  ...   2611   
12  1907  2615  9945  1314  2114  2609  1342  2496  2545  1527  ...   2851   
13  2006  2617   NaN  1338  2301  2611  1402  2597  2609  1530  ...   3167   
14  2015  2816   NaN  1342  2390  2851  1409  2615  2611  1558  ...   5215   
15  2020  3005   NaN  1402  2430  3167  1438  2809  2851  1582  ...   5471   
16  2028  4557   NaN  1409  2441  3209  1452  2816  3167  1583  ...   5546   
17  2034  5225   NaN  1438  2459  3557  1457  2820  3209  1598  ...   6655   
18  2069  5283   NaN  1457  2461  5215  1470  2881  3679  1701  ...   6807   
19  2103  5608   NaN  1470  2477  5471  1473  2882  5215  1708  ...   8404   
20  2108  6116   NaN  1473  2478  5546  1476  2883  5471  1720  ...   8481   
21  2227  6128   NaN  1474  2488  6807  1517  2905  5546  1734  ...   9945   
22  2506  6671   NaN  1513  2496  8404  1527  3048  6655  1735  ...    NaN   
23  2606  6691   NaN  1514  2597  9945  1530  3413  6807  1752  ...    NaN   
24  2612  6790   NaN  1517  2615   NaN  1531  3588  8404  1773  ...    NaN   
25  2614  8482   NaN  1527  2617   NaN  1558  3596  9945  1776  ...    NaN   
26  2838   NaN   NaN  1530  2809   NaN  1582  3702   NaN  1817  ...    NaN   
27  2852   NaN   NaN  1531  2816   NaN  1583  4557   NaN  1904  ...    NaN   
28  2904   NaN   NaN  1558  2820   NaN  1598  4927   NaN  1907  ...    NaN   
29  2915   NaN   NaN  1582  2881   NaN  1605  4930   NaN  2010  ...    NaN   
30  3050   NaN   NaN  1583  2882   NaN  1609  5283   NaN  2020  ...    NaN   
31  3669   NaN   NaN  1587  2883   NaN  1626  5306   NaN  2028  ...    NaN   
32  3704   NaN   NaN  1598  2905   NaN  1701  5522   NaN  2034  ...    NaN   
33  4555   NaN   NaN  1605  3005   NaN  1707  5608   NaN  2059  ...    NaN   
34  4770   NaN   NaN  1609  3014   NaN  1708  6116   NaN  2062  ...    NaN   
35  4999   NaN   NaN  1626  3048   NaN  1717  6128   NaN  2069  ...    NaN   
36  5007   NaN   NaN  1701  3413   NaN  1720  6456   NaN  2103  ...    NaN   
37  5538   NaN   NaN  1702  3481   NaN  1722  6581   NaN  2106  ...    NaN   
38  5906   NaN   NaN  1707  3588   NaN  1725  6671   NaN  2107  ...    NaN   
39  6183   NaN   NaN  1708  3596   NaN  1727  6768   NaN  2227  ...    NaN   
40  6243   NaN   NaN  1717  3702   NaN  1731  6790   NaN  2250  ...    NaN   
41  6464   NaN   NaN  1720  4440   NaN  1733  8114   NaN  2313  ...    NaN   
42  6585   NaN   NaN  1722  4557   NaN  1734  8482   NaN  2337  ...    NaN   
43  6834   NaN   NaN  1725  4915   NaN  1735   NaN   NaN  2354  ...    NaN   
44  8210   NaN   NaN  1727  4927   NaN  1752   NaN   NaN  2357  ...    NaN   
45  8442   NaN   NaN  1731  4930   NaN  1773   NaN   NaN  2360  ...    NaN   
46  9904   NaN   NaN  1734  4935   NaN  1776   NaN   NaN  2362  ...    NaN   
47  9926   NaN   NaN  1735  4942   NaN  1786   NaN   NaN  2374  ...    NaN   
48  9931   NaN   NaN  1752  5225   NaN  1817   NaN   NaN  2383  ...    NaN   
49  9937   NaN   NaN  1776

In [824]:
Result_TPEx

<7MA             <14MA             <30MA             <60MA  ... <180MA  \
      >3    >6    >9    >3    >6    >9    >3    >6    >9    >3  ...     >9   
0   1259  3093  5015  1240  1580  3484  1240  1599  3465  1240  ...   3465   
1   1570  3265  8941  1259  1815  4171  1259  1815  3484  1259  ...   3484   
2   2061  3349   NaN  1565  2063  4716  1570  2063  4171  1593  ...   4171   
3   2235  3570   NaN  1570  2641  4736  1593  3147  4736  1777  ...   4716   
4   2640  4123   NaN  1593  3147  5015  1777  3163  5015  1788  ...   4736   
5   2937  5278   NaN  1595  3163  6195  1785  3221  6111  2061  ...   5015   
6   3086  5356   NaN  1777  3213  8924  1788  3349  6150  2070  ...   6111   
7   3226  5520   NaN  1785  3221  8941  2061  3373  8941  2235  ...   6171   
8   3426  5523   NaN  2061  3264   NaN  2064  3444   NaN  2937  ...    NaN   
9   3516  6126   NaN  2064  3265   NaN  2070  3537   NaN  3086  ...    NaN   
10  3551  6180   NaN  2070  3293   NaN  2235  3580   NaN  3226  ...    NaN   
11  3581  6506   NaN  2235  3303   NaN  2937  3709   NaN  3323  ...    NaN   
12  3624  6629   NaN  2937  3349   NaN  3086  4123   NaN  3332  ...    NaN   
13  4114  8071   NaN  3086  3373   NaN  3141  4568   NaN  3388  ...    NaN   
14  4183  8083   NaN  3141  3531   NaN  3226  4951   NaN  3426  ...    NaN   
15  4506  8109   NaN  3169  3537   NaN  3227  5014   NaN  3577  ...    NaN   
16  4535  8342   NaN  3226  3684   NaN  3323  5278   NaN  3581  ...    NaN   
17  4561  8420   NaN  3227  3709   NaN  3332  5299   NaN  4105  ...    NaN   
18  4711  8435   NaN  3323  4123   NaN  3360  5356   NaN  4111  ...    NaN   
19  4729  8942   NaN  3332  4554   NaN  3372  5410   NaN  4114  ...    NaN   
20  5013   NaN   NaN  3360  4923   NaN  3388  5443   NaN  4116  ...    NaN   
21  5201   NaN   NaN  3372  4933   NaN  3390  5464   NaN  4121  ...    NaN   
22  5212   NaN   NaN  3388  4951   NaN  3426  5520   NaN  4160  ...    NaN   
23  5220   NaN   NaN  3390  4974   NaN  3516  5523   NaN  4173  ...    NaN   
24  5432   NaN   NaN  3426  5014   NaN  3548  6180   NaN  4183  ...    NaN   
25  5483   NaN   NaN  3441  5213   NaN  3551  6204   NaN  4413  ...    NaN   
26  5493   NaN   NaN  3548  5278   NaN  3577  6494   NaN  4433  ...    NaN   
27  5530   NaN   NaN  3551  5356   NaN  3581  6506   NaN  4528  ...    NaN   
28  5543   NaN   NaN  3577  5410   NaN  4107  6523   NaN  4535  ...    NaN   
29  5603   NaN   NaN  3581  5443   NaN  4114  6530   NaN  4538  ...    NaN   
30  6208   NaN   NaN  3624  5464   NaN  4116  6609   NaN  4561  ...    NaN   
31  6210   NaN   NaN  4107  5520   NaN  4121  6629   NaN  4584  ...    NaN   
32  6432   NaN   NaN  4114  5523   NaN  4160  6640   NaN  4711  ...    NaN   
33  6615   NaN   NaN  4116  6126   NaN  4163  8071   NaN  4735  ...    NaN   
34  6616   NaN   NaN  4160  6180   NaN  4173  8083   NaN  4744  ...    NaN   
35  6624   NaN   NaN  4163  6204   NaN  4183  8342   NaN  4931  ...    NaN   
36  6732   NaN   NaN  4173  6292   NaN  4413  8358   NaN  5013  ...    NaN   
37  7556   NaN   NaN  4183  6470   NaN  4433  8433   NaN  5201  ...    NaN   
38  8043   NaN   NaN  4413  6494   NaN  4510  8928   NaN  5202  ...    NaN   
39  8182   NaN   NaN  4433  6506   NaN  4528   NaN   NaN  5230  ...    NaN   
40  8279   NaN   NaN  4506  6523   NaN  4535   NaN   NaN  5245  ...    NaN   
41  8383   NaN   NaN  4510  6530   NaN  4538   NaN   NaN  5432  ...    NaN   
42  8415   NaN   NaN  4528  6609   NaN  4561   NaN   NaN  5483  ...    NaN   
43  8423   NaN   NaN  4535  6629   NaN  4584   NaN   NaN  5543  ...    NaN   
44  8432   NaN   NaN  4541  6640   NaN  4711   NaN   NaN  5603  ...    NaN   
45  8921   NaN   NaN  4558  6642   NaN  4735   NaN   NaN  5604  ...    NaN   
46  8923   NaN   NaN  4561  8071   NaN  4741   NaN   NaN  6173  ...    NaN   

   <360MA             <720MA             <1080MA              
       >3    >6    >9     >3    >6    >9      >3    >6    >9  
0    1240  1599  3455   1240  1569  3465    1240  1569 

In [706]:
Result_TWSE.to_excel('Result_TWSE.xlsx')
Result_TPEx.to_excel('Result_TPEx.xlsx')

In [850]:
def highlight(series):
    return ['color: red' if value > (series.mean()*3)
            else 'color: orange' if value > (series.mean()*2)
            else None for value in series]

In [837]:
temp

,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,...,9939,9940,9941,9942,9943,9944,9945,9946,9955,9958
0,14693838.0,3096389.0,943471.0,362347.0,1846185.0,51466.0,92058.0,2817255.0,26202.0,5406274.0,...,4757622.0,132925.0,1343710.0,273408.0,119553.0,110423.0,4651591.0,31892.0,364569.0,626612.0
1,12088991.0,4680929.0,661549.0,610001.0,1848681.0,112011.0,78138.0,919379.0,28482.0,6364371.0,...,2806831.0,276131.0,1422129.0,340386.0,101149.0,95547.0,4714750.0,54401.0,233565.0,1724051.0
2,12798027.0,3220723.0,363218.0,304964.0,1136083.0,164690.0,43030.0,661655.0,9987.0,4361257.0,...,1254723.0,153294.0,1974235.0,583558.0,56958.0,78249.0,4921281.0,123398.0,179500.0,2946168.0
3,21791272.0,13049631.0,604120.0,690120.0,1452002.0,170289.0,104110.0,629839.0,59599.0,2607566.0,...,1935331.0,338963.0,962121.0,193720.0,48993.0,56417.0,3393838.0,103801.0,399711.0,1551466.0
4,23896022.0,5116348.0,910917.0,880359.0,1110279.0,139299.0,192975.0,603395.0,8427.0,2047295.0,...,3334707.0,382426.0,1109706.0,490545.0,99890.0,84354.0,4633145.0,24077.0,447622.0,2180116.0
5,25192336.0,5923474.0,1036574.0,936963.0,1468308.0,239131.0,74002.0,762068.0,32632.0,2653165.0,...,3544444.0,384206.0,2297332.0,530555.0,195351.0,105202.0,7623491.0,71348.0,388904.0,4334124.0
6,22518920.0,5979885.0,653672.0,1045984.0,2377584.0,172821.0,296004.0,668133.0,25889.0,2061363.0,...,3835048.0,160304.0,1847858.0,387582.0,84615.0,91642.0,5914345.0,110518.0,6518949.0,3695158.0
7,17569854.0,5799500.0,227152.0,1080348.0,1401122.0,150669.0,69007.0,168754.0,28635.0,1736022.0,...,2587631.0,165028.0,1898842.0,122623.0,71445.0,97361.0,5963887.0,68024.0,3366629.0,2237104.0
8,15657070.0,4680549.0,385316.0,2814866.0,1038665.0,221369.0,155008.0,336614.0,34623.0,1487089.0,...,1711663.0,195984.0,1015351.0,210887.0,68676.0,53205.0,8274915.0,54902.0,10622057.0,3546992.0
9,20235848.0,5009946.0,354765.0,1940150.0,1545559.0,205226.0,75000.0,551531.0,31264.0,1364029.0,...,2227778.0,188011.0,1914560.0,311835.0,74717.0,59358.0,69798104.0,42560.0,3403275.0,2192722.0


In [848]:
list(Stock_TWSE_recentvolume.columns)

['1101',
 '1102',
 '1103',
 '1104',
 '1108',
 '1109',
 '1110',
 '1201',
 '1203',
 '1210',
 '1213',
 '1215',
 '1216',
 '1217',
 '1218',
 '1219',
 '1220',
 '1225',
 '1227',
 '1229',
 '1231',
 '1232',
 '1233',
 '1234',
 '1235',
 '1236',
 '1256',
 '1301',
 '1303',
 '1304',
 '1305',
 '1307',
 '1308',
 '1309',
 '1310',
 '1312',
 '1313',
 '1314',
 '1315',
 '1316',
 '1319',
 '1321',
 '1323',
 '1324',
 '1325',
 '1326',
 '1337',
 '1338',
 '1339',
 '1340',
 '1341',
 '1342',
 '1402',
 '1409',
 '1410',
 '1413',
 '1414',
 '1416',
 '1417',
 '1418',
 '1419',
 '1423',
 '1432',
 '1434',
 '1435',
 '1436',
 '1437',
 '1438',
 '1439',
 '1440',
 '1441',
 '1442',
 '1443',
 '1444',
 '1445',
 '1446',
 '1447',
 '1449',
 '1451',
 '1452',
 '1453',
 '1454',
 '1455',
 '1456',
 '1457',
 '1459',
 '1460',
 '1463',
 '1464',
 '1465',
 '1466',
 '1467',
 '1468',
 '1470',
 '1471',
 '1472',
 '1473',
 '1474',
 '1475',
 '1476',
 '1477',
 '1503',
 '1504',
 '1506',
 '1512',
 '1513',
 '1514',
 '1515',
 '1516',
 '1517',
 '1519',
 